# Import Packages

In [414]:
# Kyle
import pandas as pd
import numpy as np
import openpyxl as op
from datetime import datetime as dt
import re

# Load the Data

In [415]:
df_usercreditcard = pd.read_pickle('user_credit_card.pickle')
df_userdata = pd.read_json('user_data.json')
df_userjob = pd.read_csv('user_job.csv')

In [416]:
# user_credit_card.pickle
df_usercreditcard.tail()

,user_id,name,credit_card_number,issuing_bank
4995,USER49969,Kendall Waelchi,1870581879,chinabank
4996,USER57209,Lucious Kshlerin,1139377065,mayabank
4997,USER36424,Cordie Jenkins,2187584552,robinsonsbank
4998,USER45166,Tyrel Feil,2542420860,bdo
4999,USER54154,Betty Zieme,4104712921,securitybank


In [417]:
# user_data.json
df_userdata.tail()

,user_id,creation_date,name,street,state,city,country,birthdate,gender,device_address,user_type
4995,USER49969,2021-11-18 19:04:30,Kendall Waelchi,792 North Manors haven,Florida,San Diego,Jersey,1996-03-11 09:05:32,female,8e:57:7e:62:90:b4,basic
4996,USER57209,2022-03-27 09:17:43,Lucious Kshlerin,48282 Burgs chester,Kentucky,Garland,Ethiopia,2013-07-07 08:53:27,female,00:35:b5:e4:e1:aa,basic
4997,USER36424,2022-10-05 21:36:56,Cordie Jenkins,8928 North Mountains haven,North Dakota,Scottsdale,Tonga,1991-05-18 05:07:36,male,0b:0d:57:d6:c1:ab,basic
4998,USER45166,2020-07-20 03:51:41,Tyrel Feil,920 New Manor port,Rhode Island,Arlington,Virgin Islands (U.S.),1975-12-19 02:27:16,male,83:c9:1e:4e:52:f4,premium
4999,USER54154,2020-05-13 08:43:26,Betty Zieme,19808 North Pass side,Colorado,Detroit,Zambia,2010-02-21 02:56:06,female,4c:fb:dc:fb:b7:e7,premium


In [418]:
# user_job.csv
df_userjob.tail()

,Unnamed: 0,user_id,name,job_title,job_level
4995,4995,USER49969,Kendall Waelchi,Facilitator,Directives
4996,4996,USER57209,Lucious Kshlerin,Student,NaN
4997,4997,USER36424,Cordie Jenkins,Representative,Directives
4998,4998,USER45166,Tyrel Feil,Supervisor,Metrics
4999,4999,USER54154,Betty Zieme,Student,NaN


# Data Cleaning (user_credit_card.pickle)

In [419]:
# user_credit_card.pickle
df_usercreditcard

,user_id,name,credit_card_number,issuing_bank
0,USER40678,Zion Feest,4294956114,bpi
1,USER08728,Kattie Bergstrom,2742902159,bdo
2,USER29759,Aiden Corwin,1917471950,bdo
3,USER16806,Vince Gislason,3290792253,chinabank
4,USER27644,Adele Okuneva,2313395832,chinabank
...,...,...,...,...
4995,USER49969,Kendall Waelchi,1870581879,chinabank
4996,USER57209,Lucious Kshlerin,1139377065,mayabank
4997,USER36424,Cordie Jenkins,2187584552,robinsonsbank
4998,USER45166,Tyrel Feil,2542420860,bdo


In [420]:
# 1. Check for missing values
missing_values = df_usercreditcard.isnull().sum
missing_values

<bound method DataFrame.sum of       user_id   name  credit_card_number  issuing_bank
0       False  False               False         False
1       False  False               False         False
2       False  False               False         False
3       False  False               False         False
4       False  False               False         False
...       ...    ...                 ...           ...
4995    False  False               False         False
4996    False  False               False         False
4997    False  False               False         False
4998    False  False               False         False
4999    False  False               False         False

[5000 rows x 4 columns]>

In [421]:
# 2. Verify data types
data_types = df_usercreditcard.dtypes
df_usercreditcard.dtypes

user_id               object
name                  object
credit_card_number     int64
issuing_bank          object
dtype: object

In [422]:
# 3. Drop duplicates
# Finding - no duplicate data
df_usercreditcard = df_usercreditcard.drop_duplicates()
df_usercreditcard.drop_duplicates()

,user_id,name,credit_card_number,issuing_bank
0,USER40678,Zion Feest,4294956114,bpi
1,USER08728,Kattie Bergstrom,2742902159,bdo
2,USER29759,Aiden Corwin,1917471950,bdo
3,USER16806,Vince Gislason,3290792253,chinabank
4,USER27644,Adele Okuneva,2313395832,chinabank
...,...,...,...,...
4995,USER49969,Kendall Waelchi,1870581879,chinabank
4996,USER57209,Lucious Kshlerin,1139377065,mayabank
4997,USER36424,Cordie Jenkins,2187584552,robinsonsbank
4998,USER45166,Tyrel Feil,2542420860,bdo


In [423]:
# 4. Verify banks
# Finding - all rows correspond to the array of issuing banks
df_usercreditcard['issuing_bank'].unique()

array(['bpi', 'bdo', 'chinabank', 'metrobank', 'mayabank',
       'robinsonsbank', 'securitybank', 'eastwest'], dtype=object)

In [424]:
# 5. Verify credit card numbers - NOT YET CLEANED
# Finding - all credit card nos. are unique
df_usercreditcard['credit_card_number'] = df_usercreditcard['credit_card_number'].astype(str)
valid_credit_card_numbers = df_usercreditcard[df_usercreditcard['credit_card_number'].str.len() == 10]
invalid_credit_card_numbers = df_usercreditcard[df_usercreditcard['credit_card_number'].str.len() != 10]

valid_credit_card_numbers
# == 10 = 3885

invalid_credit_card_numbers
# != 10 = 1115 rows
# == 9 = 1017 rows
# == 8 = 87 rows
# == 7 = 10 rows
# == 5 = 1 row

,user_id,name,credit_card_number,issuing_bank
10,USER02300,Aubrey Mitchell,779912290,metrobank
12,USER15171,Junius Watsica,489754926,mayabank
15,USER29028,Cali Walker,786901098,securitybank
23,USER23143,Dora Jacobs,660162130,eastwest
38,USER24767,Pink Schmitt,276041500,securitybank
...,...,...,...,...
4976,USER04133,Kenyon King,629941793,securitybank
4977,USER24235,Calista Prohaska,633907739,securitybank
4978,USER05332,Benton Kassulke,195020692,eastwest
4985,USER44983,Rahul Olson,106565871,metrobank


In [425]:
# 6. Verify user_ids - NOT YET CLEANED

duplicate_user_ids = df_usercreditcard[df_usercreditcard.duplicated(subset='user_id', keep=False)]
duplicate_user_ids

,user_id,name,credit_card_number,issuing_bank
15,USER29028,Cali Walker,786901098,securitybank
38,USER24767,Pink Schmitt,276041500,securitybank
58,USER48388,Raina McLaughlin,1833621157,securitybank
65,USER34332,Keagan Feeney,1287042641,securitybank
77,USER10122,Dasia Swift,868217260,eastwest
...,...,...,...,...
4950,USER23705,Alexandria Hilpert,1171563155,robinsonsbank
4976,USER04133,Kenyon King,629941793,securitybank
4983,USER30174,Kelton Ullrich,2548987246,chinabank
4997,USER36424,Cordie Jenkins,2187584552,robinsonsbank


In [426]:
# 7. Verify names - NOT YET CLEANED
duplicate_names = df_usercreditcard[df_usercreditcard.duplicated(subset='name', keep=False)]
duplicate_names

,user_id,name,credit_card_number,issuing_bank
150,USER56658,Kathryne Bosco,359494374,mayabank
1010,USER28790,Avery Reilly,952044179,metrobank
1130,USER18695,Juwan Hintz,3688342974,bdo
1200,USER36608,Juwan Hintz,2065216590,eastwest
1853,USER05951,Juanita Beahan,1633670600,metrobank
1993,USER10216,Kody Rempel,3141144368,mayabank
2036,USER45050,Nya Abernathy,2859806841,bdo
2293,USER44994,Nya Abernathy,138972125,metrobank
2673,USER37395,Juanita Beahan,3527602678,metrobank
2940,USER64643,Avery Reilly,3157670654,mayabank


# Data Cleaning (user_data.json)

In [427]:
# user_data.json
df_userdata.head()

,user_id,creation_date,name,street,state,city,country,birthdate,gender,device_address,user_type
0,USER40678,2021-03-17 22:56:13,Zion Feest,14938 West Trace side,New Jersey,Birmingham,Hong Kong,1998-04-06 05:29:37,male,17:fb:f2:60:94:4b,basic
1,USER08728,2022-10-10 12:53:20,Kattie Bergstrom,4476 West Haven fort,Alabama,Irvine,Mayotte,2003-05-22 11:16:19,male,b0:17:a7:0b:d6:67,premium
2,USER29759,2020-05-20 04:34:44,Aiden Corwin,59980 North Crest chester,North Carolina,Tampa,Iraq,2008-08-29 16:42:05,female,24:f2:0b:88:2f:bd,basic
3,USER16806,2021-05-28 07:36:30,Vince Gislason,541 Radial mouth,Illinois,Orlando,New Zealand,2012-02-09 14:12:37,male,a4:f5:fd:fe:07:f9,basic
4,USER27644,2023-03-16 19:25:35,Adele Okuneva,896 Glen bury,Arizona,Reno,Mexico,1976-10-13 00:53:54,male,ac:80:b3:bc:8d:5f,premium


In [428]:
# 1. Check for missing values
# Finding - no missing values
ucc_missing_values = df_userdata.isna().sum()
ucc_missing_values

user_id           0
creation_date     0
name              0
street            0
state             0
city              0
country           0
birthdate         0
gender            0
device_address    0
user_type         0
dtype: int64

In [429]:
# 2. Convert the `birthdate` column to a datetime object
df_userdata['birthdate'] = pd.to_datetime(df_userdata['birthdate'])

# 2.1. Convert the 'creation_date" column to a datetime object
df_userdata['creation_date'] = pd.to_datetime(df_userdata['birthdate'])

In [430]:
# 3. Calculate the age of each user
df_userdata['age'] = df_userdata['birthdate'].apply(lambda x: x.today().year - x.year)

# 3.1. Verify if users age is in between 1 to 100.
valid_age_mask = (df_userdata['age'] >= 1) & (df_userdata['age'] <= 53) # users are aged 4 - 53
valid_age_df = df_userdata[valid_age_mask]
print("Number of users with valid age (1-100):", len(valid_age_df))

Number of users with valid age (1-100): 5000


In [431]:
# 4. Convert the `gender` column to lowercase
df_userdata['gender'] = df_userdata['gender'].str.lower()

In [432]:
# 5. Remove any spaces from the `user_type` column
df_userdata['user_type'] = df_userdata['user_type'].str.replace(' ', '')

In [433]:
# 6. Separate creation_date's date and time, to only contain the date
df_userdata['creation_date'] = pd.to_datetime(df_userdata['creation_date']).dt.date
df_userdata['birthdate'] = pd.to_datetime(df_userdata['birthdate']).dt.date

# 6.1. Verify creation_date if less than current date.
today = dt.today().date()
valid_creation_mask = df_userdata['creation_date'] < today
valid_creation_df = df_userdata[valid_creation_mask]
print(f"Number of users with valid creation date (<{today}): {len(valid_creation_df)}")

Number of users with valid creation date (<2023-12-07): 5000


In [434]:
# 7. Clean the street, state, city, and country column
# Finding - the states and cities are from the USA
# Finding - street, state, and city have completely random relationships

# 7.1. Indentify unique values of street, state, city, and country
possible_streets = df_userdata["street"].unique().tolist()
possible_states = df_userdata["state"].unique().tolist()
possible_cities = df_userdata["city"].unique().tolist()
possible_countries = df_userdata["country"].unique().tolist()

# 7.2. Convert valid countries to US, if City is part of USA
for index, row in df_userdata.iterrows():
    # Check if the city is in the USA cities list
    if row["city"] in possible_cities:
        # Update the country to "United States of America"
        df_userdata.loc[index, "country"] = "United States of America"

# 7.3. Drop 'country' since all users are from USA. PARANG I WANT TO DROP STREET AND STATE !!!
df_userdata = df_userdata.drop(columns=["country"])

In [435]:
# 8. Verify user_type.
possible_user_types = df_userdata["user_type"].unique().tolist() # basic, premium, verified

# 8.1. Check all valid user_types.
if possible_user_types:
    print("All rows have a valid user type.")
else:
    print("Some rows have invalid user types.")

All rows have a valid user type.


In [436]:
# 9. Verify user_id.
# Finding - there are duplicate user_id values
is_unique_user_id = df_userdata['user_id'].is_unique

if is_unique_user_id:
    print("All user_id values are unique.")
else:
    print("There are duplicate user_id values.")

# 9.1. Identify duplicate user_id values, then keep first instance of user_id.
df_userdata = df_userdata.drop_duplicates(subset='user_id', keep='first')

# 9.2. Identify duplicate names.
duplicate_names = df_userdata.loc[df_userdata['name'].duplicated()].index #Index([1200, 2293, 2673, 2940, 3382, 3492, 4839, 4910], dtype='int64')

df_userdata = df_userdata.drop_duplicates(subset='name', keep='first')

remaining_duplicates = df_userdata['name'].duplicated().sum()

if remaining_duplicates > 0:
  print("Warning: There are still", remaining_duplicates, "duplicate names after dropping.")
else:
  print("All duplicate names have been dropped.")

There are duplicate user_id values.
All duplicate names have been dropped.


In [437]:
df_userdata

,user_id,creation_date,name,street,state,city,birthdate,gender,device_address,user_type,age
0,USER40678,1998-04-06,Zion Feest,14938 West Trace side,New Jersey,Birmingham,1998-04-06,male,17:fb:f2:60:94:4b,basic,25
1,USER08728,2003-05-22,Kattie Bergstrom,4476 West Haven fort,Alabama,Irvine,2003-05-22,male,b0:17:a7:0b:d6:67,premium,20
2,USER29759,2008-08-29,Aiden Corwin,59980 North Crest chester,North Carolina,Tampa,2008-08-29,female,24:f2:0b:88:2f:bd,basic,15
3,USER16806,2012-02-09,Vince Gislason,541 Radial mouth,Illinois,Orlando,2012-02-09,male,a4:f5:fd:fe:07:f9,basic,11
4,USER27644,1976-10-13,Adele Okuneva,896 Glen bury,Arizona,Reno,1976-10-13,male,ac:80:b3:bc:8d:5f,premium,47
...,...,...,...,...,...,...,...,...,...,...,...
4993,USER03479,1991-02-15,Bailey Stanton,5075 Forges furt,Washington,Baton Rouge,1991-02-15,male,82:59:4e:a4:8e:c9,basic,32
4994,USER55414,2010-10-14,Gregory Hand,8489 Lake Square fort,Rhode Island,Fresno,2010-10-14,female,99:5c:72:9e:e3:0a,basic,13
4995,USER49969,1996-03-11,Kendall Waelchi,792 North Manors haven,Florida,San Diego,1996-03-11,female,8e:57:7e:62:90:b4,basic,27
4996,USER57209,2013-07-07,Lucious Kshlerin,48282 Burgs chester,Kentucky,Garland,2013-07-07,female,00:35:b5:e4:e1:aa,basic,10


In [438]:
# 10. Check for rows with null values. NEED HELP BAKA NAKA NULL OR NaN
null_rows = df_userdata.isnull().any(axis=1)
num_null_rows = null_rows.sum()

if num_null_rows > 0:
  print(f"There are {num_null_rows} rows with null values.")

  # Optionally, get the rows with null values
  null_df = df_userdata[null_rows]

  # Print the first 5 rows with null values
  print(null_df.head())
else:
  print("There are no rows with null values.")

There are no rows with null values.


# Data Cleaning (user_job.csv)

In [439]:
df_userjob

,Unnamed: 0,user_id,name,job_title,job_level
0,0,USER40678,Zion Feest,Technician,Accounts
1,1,USER08728,Kattie Bergstrom,Technician,Solutions
2,2,USER29759,Aiden Corwin,Student,NaN
3,3,USER16806,Vince Gislason,Student,NaN
4,4,USER27644,Adele Okuneva,Associate,Usability
...,...,...,...,...,...
4995,4995,USER49969,Kendall Waelchi,Facilitator,Directives
4996,4996,USER57209,Lucious Kshlerin,Student,NaN
4997,4997,USER36424,Cordie Jenkins,Representative,Directives
4998,4998,USER45166,Tyrel Feil,Supervisor,Metrics


In [446]:
# 1. Verify job_title
# Finding - missing_values = 0
# unique_job_titles = ['Technician', 'Student', 'Associate', 'Liaison', 'Director',
       # 'Producer', 'Executive', 'Strategist', 'Planner', 'Facilitator',
       # 'Designer', 'Orchestrator', 'Developer', 'Manager', 'Consultant',
       # 'Engineer', 'Assistant', 'Supervisor', 'Representative',
       # 'Specialist', 'Analyst', 'Coordinator', 'Architect', 'Agent',
       # 'Officer', 'Administrator']

unique_job_titles = df_userjob['job_title'].unique()
missing_values = df_userjob['job_title'].isnull().sum()

In [455]:
# 2. Verify job_level
# Finding - unique jobs include NaN
# Finding - 1629 missing values in job_level

unique_job_levels = df_userjob['job_level'].unique()
missing_values_job_level = df_userjob['job_level'].isnull().sum()

missing_values_job_level

0

In [457]:
# 3. Convert name, job_title, and job_level to string
df_userjob['name'] = df_userjob['name'].astype(str)
df_userjob['job_title'] = df_userjob['job_title'].astype(str)
df_userjob['job_level'] = df_userjob['job_level'].astype(str)

print(df_userjob.dtypes)

Unnamed: 0     int64
user_id       object
name          object
job_title     object
job_level     object
dtype: object
